# Challenge Six: FEMA ReadyNow! Agent

In [1]:
import os
import logging
import vertexai
from pathlib import Path
from dotenv import load_dotenv
from utils.agent_callbacks import *
from utils.agent_tools import get_weather

env_path = Path.cwd().parent / '.env'
load_dotenv(env_path)

logger = logging.getLogger(__name__)
logging.basicConfig(filename="ReadyNow.log", level=logging.INFO)

GCP_PROJECT = os.getenv("GCP_PROJECT")
GCP_REGION = os.getenv("GCP_REGION")
GCP_BUCKET = os.getenv("GCP_BUCKET")
GOOGLE_MAP_KEY = os.getenv("GOOGLE_MAP_KEY")

vertexai.init(project=GCP_PROJECT, location=GCP_REGION, staging_bucket=GCP_BUCKET)

In [4]:
from google.adk.agents import LlmAgent, Agent, SequentialAgent
from google.adk.tools import google_search
from vertexai.preview import reasoning_engines
from IPython.display import Markdown, display

weather_agent = LlmAgent(
    name="weather_agent_v2",
    model="gemini-2.0-flash", # Can be a string for Gemini or a LiteLlm object
    description="Provides weather information for specific cities.",
    instruction="You are a helpful weather assistant. "
                "When the user asks for the weather in a specific city, "
                "use the 'get_weather' tool to find the information. "
                "If the tool returns an error, inform the user politely. "
                "If the tool is successful, present the weather report clearly as a markdown table.",
    tools=[get_weather], 
    output_key="weather_output",
    before_model_callback=chained_before_callback,
    after_model_callback=model_response_log_callback
)

search_agent = LlmAgent(
    name="search_agent",
    model="gemini-2.0-flash",
    instruction="Use the Google Search tool to find information about the user question in the state key 'user_query' from the context that you are a FEMA agent trying to determine news and emergency information.",
    tools=[google_search],
    output_key="search_results",
)

critique_agent = LlmAgent(
    name="critique_agent",
    model="gemini-2.0-flash",
    instruction="Critique the search results stored in the state key 'search_results'. Your purpose is to create better search results based on the fact that you are trying to determine emergency/disaster news and evacuation plans.",
    output_key="critique_results",
)

refine_agent = LlmAgent(
    name="refine_agent",
    model="gemini-2.5-flash",
    instruction="Refine the search results based on the critique stored in the state key 'critique_results' using the Google Search tool",
    tools=[google_search],
    output_key="refined_results",
)

presentation_agent = LlmAgent(
    name="presentation_agent",
    model="gemini-2.0-flash",
    instruction="Take the state keys 'refined_results' and 'weather_output' and format it so that it can be used as the final output to an end user.",
)

greeter_agent = Agent(
    name="greeter_agent",
    model="gemini-2.0-flash",
    instruction="You are a FEMA emergency response agent. Take the user query, then pass to 'search_agent'.",
    output_key="user_query"
)

search_sequence = SequentialAgent(
    name="search_sequence",
    sub_agents=[greeter_agent, weather_agent, search_agent, critique_agent, refine_agent, presentation_agent],
)

In [5]:
app = reasoning_engines.AdkApp(
    agent=search_sequence,
    enable_tracing=False,
)

user_id = "test-user-id"
session = app.create_session(user_id=user_id)

In [7]:
for event in app.stream_query(
    user_id=user_id,
    session_id = session.id,
    message="I'm in Kerr County, Texas and flooding is happening please help me.",
):
  lastevent = event

display(Markdown(lastevent["content"]["parts"][0]["text"]))

/workspaces/genai-skills-workshop-shawn-mittal/venv/lib/python3.12/site-packages/vertexai/generative_models/_generative_models.py:433: UserWarning: This feature is deprecated as of June 24, 2025 and will be removed on June 24, 2026. For details, see https://cloud.google.com/vertex-ai/generative-ai/docs/deprecations/genai-vertexai-sdk.
  warning_logs.show_deprecation_warning()


This is a critical situation. Please stay safe and take immediate action. Here's what you need to know about the flooding in Kerr County, Texas:

**IMMEDIATE ACTIONS - Your Safety is the Priority**

*   **Move to Higher Ground IMMEDIATELY:** If you are near any creek, stream, or the Guadalupe River, seek higher ground immediately. **Do not attempt to travel through floodwaters** on foot or in a vehicle. The water is dangerous, and roads may be gone.
*   **Shelter in Place:** If you cannot evacuate safely, find the highest possible location inside the building you are in.
*   **Road Closures:** Many roads are closed due to flooding. Stay off the roads. Avoid the affected areas so rescue workers can do their job.
*   **Alerts May Be Delayed:** Do not wait for an official alert if you feel you are in danger. Kerr County does not have a river warning system.

**The Situation Right Now (July 9-10, 2025)**

*   Kerr County is in a state of disaster.
*   The Guadalupe River crested beyond 1987 flood levels.
*   Tragically, 96 people have died in Kerr County and 119 people have died in Central Texas due to the flooding.
*   Search and rescue efforts are ongoing with more than 2,100 people involved. Focus is heavily on an 8-mile area of the Guadalupe River near Ingram.
*   Power is restored to most areas.
*   161 people are still missing in Kerr County.

**Specific Area Information**

*   **Ingram:** Search and rescue is focused on the Guadalupe River.
*   **Comfort:** The town benefited from its own warning system, and the community is supporting those who lost homes. Carolyn Road homes were destroyed.
*   **Kerrville:** Emergency crews rescued people from rising waters. A Disaster Recovery Center is open at First Baptist Church (625 Washington Street) in Kerrville with financial support available.

**Where to Go for Help**

*   **Shelter:** First United Methodist Church (321 Thompson Drive, Kerrville, TX).
*   **Missing Persons:** If you have information about someone missing, update it at kerrvillemissing.dps.texas.gov or call 830-258-1111.
*   **Pets:** Kerrville Pets Alive! can help with pet needs at 830-200-0539 or info@kerrvilleptsalive.org.
*   **Donations:** Donate to the American Red Cross to help those affected.

**Important Contacts**

*   **Kerr County Emergency Management:** 830-315-2430
*   **Kerr County Sheriff's Office:** 830-896-1216
*   **Sign up for Emergency Alerts:** CodeRED Emergency Notification System via the Kerr County Sheriff's Office or City of Kerrville websites.

Please, prioritize your safety and the safety of others. Stay informed and follow instructions from emergency personnel. Stay away from the flooded areas.

